# MMseqs2 Protein Clustering

This notebook demonstrates how to use MMSeqsUtils for fast protein sequence clustering.

## Overview

MMseqs2 is an ultra-fast tool for protein sequence searching and clustering. MMSeqsUtils provides:
- Protein sequence clustering with configurable parameters
- Automatic handling of FASTA file creation and parsing
- Cluster representative extraction
- Cluster membership mapping

## Prerequisites

- **MMseqs2**: Install via conda, brew, or from source
  ```bash
  conda install -c conda-forge -c bioconda mmseqs2
  # or
  brew install mmseqs2
  ```

## 1. Setup

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Project root: {project_root}")

## 2. Initialize MMSeqsUtils

In [ ]:
from kbutillib import MMSeqsUtils

# Initialize MMseqs2 utilities
util = MMSeqsUtils()

print(f"MMseqs2 available: {util.mmseqs_available}")
print(f"MMseqs2 executable: {util.mmseqs_executable}")

## 3. Prepare Sample Protein Data

MMSeqsUtils accepts proteins as a list of dictionaries with `id` and `protein_translation` fields:

In [ ]:
# Sample protein sequences for demonstration
# These include some identical/similar sequences that should cluster together

sample_proteins = [
    # Group 1: Identical sequences (should cluster together)
    {
        "id": "prot_001",
        "protein_translation": "MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEKAVQVKVKALPDAQFEVVHSLAKWKRQQIAAALEHHHHHH",
        "function": "Hypothetical protein A"
    },
    {
        "id": "prot_002",
        "protein_translation": "MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEKAVQVKVKALPDAQFEVVHSLAKWKRQQIAAALEHHHHHH",
        "function": "Hypothetical protein A variant"
    },
    
    # Group 2: Different protein
    {
        "id": "prot_003",
        "protein_translation": "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKL",
        "function": "Hemoglobin-like protein"
    },
    
    # Group 3: Another different protein
    {
        "id": "prot_004",
        "protein_translation": "MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRCALINMVFQMGETGVAGFTNSLRMLQQKRWDEAAVNLAKSRWYNQTPNRAKRVITTFRTGTWDAYKNL",
        "function": "DNA binding protein"
    },
    {
        "id": "prot_005",
        "protein_translation": "MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRCALINMVFQMGETGVAGFTNSLRMLQQKRWDEAAVNLAKSRWYNQTPNRAKRVITTFRTGTWDAYKNL",
        "function": "DNA binding protein variant"
    },
    
    # Additional unique proteins
    {
        "id": "prot_006",
        "protein_translation": "MGSSHHHHHHSSGLVPRGSHMRGPNPTAASLEASAGPFTVRSFTVSRPSGYGAGTVYYPTNAGGTVGAIAIVPGYTARQSSIKWWGPRLASHGFVVITIDTNSTLDQPSSRSSQQMAALRQVASLNGTSSSPIYGKVDTARMGVMGWSMGGGGSLISAANNPSLKAAAPQAPWDSSTNFSSVTVPTLIFACENDSIAPVNSSALPIYDSMSRNAKQFLEINGGSHSCANSGNSNQALIGKKGVAWMKRFMDNDTRYSTFACENPNSTRVSDFRTANCSLEDPAANKARKEAELAAATAEQ",
        "function": "Unique protein 1"
    },
    {
        "id": "prot_007",
        "protein_translation": "MKFLILLFNILCLFPVLAADNHGVGPQGASGVDPITFDINSNQTGVQLTLFRPGQNNWVTGFGGTYAGGSSASSIVGVTVSGSVTAQTYPPATVTQSQSQTSATGSVNVTVTPTTPVQTTATLQTPTTNVTGNVTGLSGTSINVTGLSTGVGGNITNSISSSTVGTGQY",
        "function": "Unique protein 2"
    },
]

print(f"Prepared {len(sample_proteins)} sample proteins")
for p in sample_proteins:
    print(f"  {p['id']}: {len(p['protein_translation'])} aa - {p['function']}")

## 4. Basic Clustering

Cluster proteins using default parameters:

In [ ]:
# Simple clustering with easy_cluster
result = util.easy_cluster(
    proteins=sample_proteins,
    min_seq_id=0.9,  # 90% sequence identity threshold
    coverage=0.8     # 80% coverage threshold
)

if result["success"]:
    print(f"Clustering successful!")
    print(f"  Input proteins: {result['num_proteins']}")
    print(f"  Clusters found: {result['num_clusters']}")
    print(f"  Singletons: {result['singletons']}")
    
    print("\nClusters:")
    for i, cluster in enumerate(result["clusters"]):
        print(f"  Cluster {i+1}: Representative={cluster['representative']}, Size={cluster['size']}")
        print(f"    Members: {', '.join(cluster['members'])}")
else:
    print(f"Clustering failed: {result['error']}")

## 5. Advanced Clustering Options

Use the full `cluster_proteins` method for more control:

In [ ]:
# Advanced clustering with more options
result = util.cluster_proteins(
    proteins=sample_proteins,
    min_seq_id=0.5,      # 50% sequence identity
    coverage=0.8,        # 80% coverage
    coverage_mode=0,     # 0: query+target, 1: target, 2: query, 3: target length
    cluster_mode=0,      # 0: set cover, 1: connected component, 2: greedy incremental
    sensitivity=4.0,     # Higher = more sensitive but slower
    threads=4            # Use 4 threads
)

if result["success"]:
    print(f"Clustering with 50% identity threshold:")
    print(f"  Clusters: {result['num_clusters']}")
    print(f"  Singletons: {result['singletons']}")
    
    print("\nParameters used:")
    for key, value in result["parameters"].items():
        print(f"  {key}: {value}")
else:
    print(f"Clustering failed: {result['error']}")

## 6. Extract Cluster Representatives

Get just the representative proteins from each cluster:

In [ ]:
# Get representative proteins (IDs only)
representatives = util.get_cluster_representatives(result)
print(f"Representative protein IDs:")
for rep in representatives:
    print(f"  {rep['id']}")

print()

# Get full protein data for representatives
representatives_full = util.get_cluster_representatives(result, proteins=sample_proteins)
print(f"Representative proteins with full data:")
for rep in representatives_full:
    print(f"  {rep['id']}: {rep['function']} ({len(rep['protein_translation'])} aa)")

## 7. Cluster Membership Mapping

Create a mapping from each protein to its cluster representative:

In [ ]:
# Get membership mapping
membership = util.get_cluster_membership(result)

print("Protein -> Cluster Representative mapping:")
for protein_id, representative_id in membership.items():
    if protein_id == representative_id:
        print(f"  {protein_id} -> {representative_id} (is representative)")
    else:
        print(f"  {protein_id} -> {representative_id}")

## 8. Working with Real Genome Data

Example of clustering proteins from a genome object:

In [ ]:
# Example: Clustering proteins from a genome (pseudo-code)
'''
from kbutillib import KBGenomeUtils

# Get genome data
genome_utils = KBGenomeUtils()
genome = genome_utils.get_genome("some_genome_ref")

# Extract proteins from genome features
proteins = []
for feature in genome.get("features", []):
    if feature.get("protein_translation"):
        proteins.append({
            "id": feature["id"],
            "protein_translation": feature["protein_translation"],
            "function": feature.get("function", "Unknown")
        })

# Cluster the proteins
result = util.cluster_proteins(proteins, min_seq_id=0.9)
print(f"Clustered {len(proteins)} proteins into {result['num_clusters']} clusters")
'''

print("Example shows how to integrate with KBGenomeUtils")
print("Uncomment and modify for your specific genome")

## 9. Configuration

MMSeqsUtils can be configured via config.yaml:

In [ ]:
# Display current configuration
print("Current MMseqs2 Configuration:")
print(f"  Executable: {util.mmseqs_executable}")
print(f"  Available: {util.mmseqs_available}")

print("\nTo customize, add to ~/.kbutillib/config.yaml:")
print("""
mmseqs:
  executable: /path/to/mmseqs
""")

## Summary

MMSeqsUtils provides:
- **Fast Clustering** - Leverage MMseqs2's speed for large protein sets
- **Flexible Parameters** - Control identity, coverage, and algorithm
- **Easy Integration** - Works with standard protein dictionaries
- **Helper Methods** - Extract representatives and membership maps

### Clustering Parameters

| Parameter | Description | Default |
|-----------|-------------|--------|
| `min_seq_id` | Minimum sequence identity (0.0-1.0) | 0.5 |
| `coverage` | Minimum coverage (0.0-1.0) | 0.8 |
| `coverage_mode` | 0=both, 1=target, 2=query, 3=target length | 0 |
| `cluster_mode` | 0=set cover, 1=connected, 2=greedy | 0 |
| `sensitivity` | Search sensitivity (higher = slower) | 4.0 |

### Next Steps
- Cluster proteins from your genome collections
- Use representatives for downstream analysis
- Combine with other KBUtilLib utilities for workflows